In [11]:
!pip install streamlit pyngrok deepface transformers torch nltk scikit-learn datasets pillow tensorflow ytmusicapi
import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')
print("NLTK ready!")
import re
import streamlit as st
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from transformers import pipeline
from datasets import load_dataset
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score
import numpy as np
from deepface import DeepFace
from PIL import Image
import io
from ytmusicapi import YTMusic  # YouTube Music lib
import random
print("✅ All imports successful! (YouTube Music ready)")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.5/100.5 kB 3.9 MB/s eta 0:00:00
NLTK ready!
✅ All imports successful! (YouTube Music ready)


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
# Load emotion dataset
emotion_dataset = load_dataset('dair-ai/emotion', split='train')
emotion_df = emotion_dataset.to_pandas()
label_map = {0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'surprise'}
emotion_df['emotion'] = emotion_df['label'].map(label_map)
emotion_df = emotion_df[['text', 'emotion']]

def preprocess_text(text):
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'\@w+|\#', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    tokens = word_tokenize(text.lower())
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

# Text emotion detection
@st.cache_resource
def load_text_model():
    return pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)

def detect_text_emotion(text):
    model = load_text_model()
    processed_text = preprocess_text(text)
    if not processed_text.strip():
        return "neutral", 0.0, {}
    results = model(processed_text)
    emotion_scores = {res['label']: res['score'] for res in results[0]}
    dominant_emotion = max(emotion_scores, key=emotion_scores.get)
    return dominant_emotion, emotion_scores[dominant_emotion], emotion_scores

# Face emotion detection
@st.cache_resource
def load_face_model():
    return DeepFace

def detect_face_emotion(image):
    try:
        # If image is bytes, convert to PIL Image
        if isinstance(image, bytes):
            image = Image.open(io.BytesIO(image)).convert('RGB')
        # DeepFace accepts PIL Image directly
        result = DeepFace.analyze(img_path=image, actions=['emotion'], enforce_detection=False)
        emotion = result[0]['dominant_emotion']
        confidence = max(result[0]['emotion'].values())
        return emotion.lower(), confidence, result[0]['emotion']
    except Exception as e:
        st.error(f"Face detection failed: {e}. Ensure a clear face in the image.")
        return "neutral", 0.0, {}

# Fallback samples
sample_data = [
    {'title': 'Happy', 'artist': 'Pharrell Williams', 'valence': 0.85, 'energy': 0.70, 'mood': 'joy'},
    {'title': "Don't Stop Believin'", 'artist': 'Journey', 'valence': 0.75, 'energy': 0.80, 'mood': 'joy'},
    {'title': 'Someone Like You', 'artist': 'Adele', 'valence': 0.30, 'energy': 0.40, 'mood': 'sadness'},
    {'title': 'Fix You', 'artist': 'Coldplay', 'valence': 0.35, 'energy': 0.50, 'mood': 'sadness'},
    {'title': 'Smells Like Teen Spirit', 'artist': 'Nirvana', 'valence': 0.25, 'energy': 0.90, 'mood': 'anger'},
    {'title': 'Bad Guy', 'artist': 'Billie Eilish', 'valence': 0.20, 'energy': 0.75, 'mood': 'anger'},
    {'title': 'Thriller', 'artist': 'Michael Jackson', 'valence': 0.40, 'energy': 0.85, 'mood': 'fear'},
    {'title': 'Every Breath You Take', 'artist': 'The Police', 'valence': 0.45, 'energy': 0.30, 'mood': 'fear'},
    {'title': 'Uptown Funk', 'artist': 'Bruno Mars', 'valence': 0.80, 'energy': 0.95, 'mood': 'surprise'},
    {'title': 'Shake It Off', 'artist': 'Taylor Swift', 'valence': 0.70, 'energy': 0.85, 'mood': 'surprise'},
    {'title': 'Perfect', 'artist': 'Ed Sheeran', 'valence': 0.90, 'energy': 0.35, 'mood': 'love'},
    {'title': 'All of Me', 'artist': 'John Legend', 'valence': 0.85, 'energy': 0.40, 'mood': 'love'},
    {'title': 'Shape of You', 'artist': 'Ed Sheeran', 'valence': 0.65, 'energy': 0.60, 'mood': 'neutral'},
    {'title': 'Blinding Lights', 'artist': 'The Weeknd', 'valence': 0.70, 'energy': 0.75, 'mood': 'neutral'},
] * 4
random.seed(42)
random.shuffle(sample_data)
music_df = pd.DataFrame(sample_data)
music_df['tag_str'] = music_df['mood'] + ' valence_' + music_df['valence'].round(2).astype(str) + ' energy_' + music_df['energy'].round(2).astype(str)

@st.cache_data
def init_recs():
    vectorizer = TfidfVectorizer(max_features=50, stop_words='english')
    tag_matrix = vectorizer.fit_transform(music_df['tag_str'])
    return vectorizer, tag_matrix

vectorizer, tag_matrix = init_recs()

def get_music_recs_fallback(emotion, top_n=3):
    emotion_queries = {
        'joy': 'joy valence_high energy_high', 'happy': 'joy valence_high energy_high',
        'sadness': 'sadness valence_low energy_low', 'sad': 'sadness valence_low energy_low',
        'anger': 'anger valence_low energy_high', 'angry': 'anger valence_low energy_high',
        'fear': 'fear valence_low energy_low',
        'surprise': 'surprise valence_high energy_high',
        'love': 'love valence_high energy_low',
        'neutral': 'neutral valence_medium energy_medium'
    }
    query_tags = emotion_queries.get(emotion, emotion_queries['neutral'])
    query_vec = vectorizer.transform([query_tags])
    sim_scores = cosine_similarity(query_vec, tag_matrix).flatten()
    top_indices = sim_scores.argsort()[-top_n:][::-1]
    recs = music_df.iloc[top_indices][['title', 'artist', 'mood', 'valence', 'energy']].to_dict('records')
    return recs

# YouTube Music Client
@st.cache_resource
def get_ytmusic_client(cookies=None):
    try:
        yt = YTMusic('headers_auth.json', auth=cookies) if cookies else YTMusic()
        return yt
    except Exception as e:
        st.error(f"YouTube Music client error: {e}")
        return None

# YouTube Recs
def get_music_recs(emotion, yt_client, top_n=3):
    if yt_client is None:
        return get_music_recs_fallback(emotion, top_n)

    emotion_queries = {
        'joy': 'joyful upbeat pop songs', 'happy': 'happy upbeat pop songs',
        'sadness': 'sad melancholic acoustic songs', 'sad': 'sad melancholic acoustic songs',
        'anger': 'angry intense rock songs', 'angry': 'angry intense rock songs',
        'fear': 'scary thriller suspense songs',
        'surprise': 'surprising energetic fun songs',
        'love': 'romantic love ballad songs',
        'neutral': 'chill relaxed ambient songs'
    }
    search_query = emotion_queries.get(emotion, 'chill relaxed songs')

    mood_targets = {
        'joy': {'valence': 0.8, 'energy': 0.7}, 'happy': {'valence': 0.8, 'energy': 0.7},
        'sadness': {'valence': 0.3, 'energy': 0.4}, 'sad': {'valence': 0.3, 'energy': 0.4},
        'anger': {'valence': 0.2, 'energy': 0.8}, 'angry': {'valence': 0.2, 'energy': 0.8},
        'fear': {'valence': 0.4, 'energy': 0.6},
        'surprise': {'valence': 0.7, 'energy': 0.9},
        'love': {'valence': 0.9, 'energy': 0.3},
        'neutral': {'valence': 0.6, 'energy': 0.6}
    }
    targets = mood_targets.get(emotion, mood_targets['neutral'])

    try:
        results = yt_client.search(search_query, filter='songs', limit=top_n)
        tracks = []
        for item in results:
            video_id = item.get('videoId')
            if video_id:
                tracks.append({
                    'title': item['title'],
                    'artist': ', '.join([a['name'] for a in item.get('artists', [])]),
                    'video_url': f"https://www.youtube.com/watch?v={video_id}",
                    'valence': targets['valence'],
                    'energy': targets['energy'],
                    'mood': emotion
                })
        if tracks:
            st.success(f"✅ Fetched {len(tracks)} YouTube Music tracks for '{emotion}' mood!")
            return tracks
        else:
            return get_music_recs_fallback(emotion, top_n)
    except Exception as e:
        st.error(f"YouTube Music search error: {e}. Using fallback.")
        return get_music_recs_fallback(emotion, top_n)

# Accuracy
@st.cache_data
def compute_acc():
    X_sample = emotion_df['text'][:100].tolist()
    y_sample = emotion_df['emotion'][:100].tolist()
    y_pred_sample = [detect_text_emotion(text)[0] for text in X_sample]
    return accuracy_score(y_sample, y_pred_sample)

acc = compute_acc()
print(f" Accuracy: {acc:.2%} | YouTube Music + Face Fix ready!")

2025-10-27 07:41:40.570 No runtime found, using MemoryCacheStorageManager
2025-10-27 07:41:40.573 No runtime found, using MemoryCacheStorageManager


✅ Accuracy: 87.00% | YouTube Music + Face Fix ready!


In [16]:
app_code = '''
import streamlit as st
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from transformers import pipeline
from datasets import load_dataset
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score
import numpy as np
from deepface import DeepFace
from PIL import Image
import io
import nltk
from ytmusicapi import YTMusic
import random

# NLTK setup
nltk.download('punkt_tab', quiet=True)
nltk.download('stopwords', quiet=True)

# Load emotion dataset
@st.cache_data
def load_emotion_data():
    emotion_dataset = load_dataset('dair-ai/emotion', split='train')
    emotion_df = emotion_dataset.to_pandas()
    label_map = {0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'surprise'}
    emotion_df['emotion'] = emotion_df['label'].map(label_map)
    return emotion_df[['text', 'emotion']]

emotion_df = load_emotion_data()

def preprocess_text(text):
    text = re.sub(r'http\\\\S+|www\\\\S+|https\\\\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'\\\\@w+|\\\\#', '', text)
    text = re.sub(r'[^a-zA-Z\\\\s]', '', text)
    tokens = word_tokenize(text.lower())
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

# Text emotion detection
@st.cache_resource
def load_text_model():
    return pipeline("text-classification",
                    model="j-hartmann/emotion-english-distilroberta-base",
                    return_all_scores=True)

def detect_text_emotion(text):
    model = load_text_model()
    processed_text = preprocess_text(text)
    if not processed_text.strip():
        return "neutral", 0.0, {}
    results = model(processed_text)
    emotion_scores = {res['label']: res['score'] for res in results[0]}
    dominant_emotion = max(emotion_scores, key=emotion_scores.get)
    return dominant_emotion, emotion_scores[dominant_emotion], emotion_scores

# Face emotion detection (FIXED: Handles bytes/PIL)
@st.cache_resource
def load_face_model():
    return DeepFace

def detect_face_emotion(image):
    try:
        # Convert bytes to PIL if needed
        if isinstance(image, bytes):
            image = Image.open(io.BytesIO(image)).convert('RGB')
        result = DeepFace.analyze(img_path=image, actions=['emotion'], enforce_detection=False)
        emotion = result[0]['dominant_emotion']
        confidence = max(result[0]['emotion'].values())
        return emotion.lower(), confidence, result[0]['emotion']
    except Exception as e:
        st.error(f"Face detection failed: {e}. Ensure a clear face in the image.")
        return "neutral", 0.0, {}

# Fallback samples
sample_data = [
    {'title': 'Happy', 'artist': 'Pharrell Williams', 'valence': 0.85, 'energy': 0.70, 'mood': 'joy'},
    {'title': "Don't Stop Believin'", 'artist': 'Journey', 'valence': 0.75, 'energy': 0.80, 'mood': 'joy'},
    {'title': 'Someone Like You', 'artist': 'Adele', 'valence': 0.30, 'energy': 0.40, 'mood': 'sadness'},
    {'title': 'Fix You', 'artist': 'Coldplay', 'valence': 0.35, 'energy': 0.50, 'mood': 'sadness'},
    {'title': 'Smells Like Teen Spirit', 'artist': 'Nirvana', 'valence': 0.25, 'energy': 0.90, 'mood': 'anger'},
    {'title': 'Bad Guy', 'artist': 'Billie Eilish', 'valence': 0.20, 'energy': 0.75, 'mood': 'anger'},
    {'title': 'Thriller', 'artist': 'Michael Jackson', 'valence': 0.40, 'energy': 0.85, 'mood': 'fear'},
    {'title': 'Every Breath You Take', 'artist': 'The Police', 'valence': 0.45, 'energy': 0.30, 'mood': 'fear'},
    {'title': 'Uptown Funk', 'artist': 'Bruno Mars', 'valence': 0.80, 'energy': 0.95, 'mood': 'surprise'},
    {'title': 'Shake It Off', 'artist': 'Taylor Swift', 'valence': 0.70, 'energy': 0.85, 'mood': 'surprise'},
    {'title': 'Perfect', 'artist': 'Ed Sheeran', 'valence': 0.90, 'energy': 0.35, 'mood': 'love'},
    {'title': 'All of Me', 'artist': 'John Legend', 'valence': 0.85, 'energy': 0.40, 'mood': 'love'},
    {'title': 'Shape of You', 'artist': 'Ed Sheeran', 'valence': 0.65, 'energy': 0.60, 'mood': 'neutral'},
    {'title': 'Blinding Lights', 'artist': 'The Weeknd', 'valence': 0.70, 'energy': 0.75, 'mood': 'neutral'},
] * 4
random.seed(42)
random.shuffle(sample_data)
music_df = pd.DataFrame(sample_data)

music_df['tag_str'] = (music_df['mood'] + ' valence_' + music_df['valence'].round(2).astype(str) +
                       ' energy_' + music_df['energy'].round(2).astype(str))

@st.cache_data
def init_recs():
    vectorizer = TfidfVectorizer(max_features=50, stop_words='english')
    tag_matrix = vectorizer.fit_transform(music_df['tag_str'])
    return vectorizer, tag_matrix

vectorizer, tag_matrix = init_recs()

def get_music_recs_fallback(emotion, top_n=3):
    emotion_queries = {
        'joy': 'joy valence_high energy_high', 'happy': 'joy valence_high energy_high',
        'sadness': 'sadness valence_low energy_low', 'sad': 'sadness valence_low energy_low',
        'anger': 'anger valence_low energy_high', 'angry': 'anger valence_low energy_high',
        'fear': 'fear valence_low energy_low',
        'surprise': 'surprise valence_high energy_high',
        'love': 'love valence_high energy_low',
        'neutral': 'neutral valence_medium energy_medium'
    }
    query_tags = emotion_queries.get(emotion, emotion_queries['neutral'])
    query_vec = vectorizer.transform([query_tags])
    sim_scores = cosine_similarity(query_vec, tag_matrix).flatten()
    top_indices = sim_scores.argsort()[-top_n:][::-1]
    recs = music_df.iloc[top_indices][['title', 'artist', 'mood', 'valence', 'energy']].to_dict('records')
    return recs

# YouTube Music Setup
@st.cache_resource
def get_ytmusic_client(cookies=None):
    try:
        yt = YTMusic('headers_auth.json', auth=cookies) if cookies else YTMusic()
        return yt
    except Exception as e:
        st.error(f"YouTube Music client error: {e}")
        return None

def get_music_recs(emotion, yt_client, top_n=3):
    if yt_client is None:
        return get_music_recs_fallback(emotion, top_n)

    emotion_queries = {
        'joy': 'joyful upbeat pop songs', 'happy': 'happy upbeat pop songs',
        'sadness': 'sad melancholic acoustic songs', 'sad': 'sad melancholic acoustic songs',
        'anger': 'angry intense rock songs', 'angry': 'angry intense rock songs',
        'fear': 'scary thriller suspense songs',
        'surprise': 'surprising energetic fun songs',
        'love': 'romantic love ballad songs',
        'neutral': 'chill relaxed ambient songs'
    }
    search_query = emotion_queries.get(emotion, 'chill relaxed songs')

    mood_targets = {
        'joy': {'valence': 0.8, 'energy': 0.7}, 'happy': {'valence': 0.8, 'energy': 0.7},
        'sadness': {'valence': 0.3, 'energy': 0.4}, 'sad': {'valence': 0.3, 'energy': 0.4},
        'anger': {'valence': 0.2, 'energy': 0.8}, 'angry': {'valence': 0.2, 'energy': 0.8},
        'fear': {'valence': 0.4, 'energy': 0.6},
        'surprise': {'valence': 0.7, 'energy': 0.9},
        'love': {'valence': 0.9, 'energy': 0.3},
        'neutral': {'valence': 0.6, 'energy': 0.6}
    }
    targets = mood_targets.get(emotion, mood_targets['neutral'])

    try:
        results = yt_client.search(search_query, filter='songs', limit=top_n)
        tracks = []
        for item in results:
            video_id = item.get('videoId')
            if video_id:
                tracks.append({
                    'title': item['title'],
                    'artist': ', '.join([a['name'] for a in item.get('artists', [])]),
                    'video_url': f"https://www.youtube.com/watch?v={video_id}",
                    'valence': targets['valence'],
                    'energy': targets['energy'],
                    'mood': emotion
                })
        if tracks:
            st.success(f"✅ Fetched {len(tracks)} YouTube Music tracks for '{emotion}' mood!")
            return tracks
        else:
            return get_music_recs_fallback(emotion, top_n)
    except Exception as e:
        st.error(f"YouTube Music search error: {e}. Using fallback.")
        return get_music_recs_fallback(emotion, top_n)

# Compute acc
@st.cache_data
def compute_acc():
    X_sample = emotion_df['text'][:100].tolist()
    y_sample = emotion_df['emotion'][:100].tolist()
    y_pred_sample = [detect_text_emotion(text)[0] for text in X_sample]
    return accuracy_score(y_sample, y_pred_sample)

acc = compute_acc()

# Streamlit UI
st.set_page_config(page_title="MoodMate", page_icon="🎵", layout="wide")

st.title("🎭 MoodMate: Emotion Detection & Music Recommender")
st.write("Detect mood via **text** or **face**, get YouTube Music recs! (Videos embed & play)")

# Sidebar: YouTube Setup (optional cookies)
with st.sidebar:
    st.header("🔑 YouTube Music Setup")
    st.info("No key needed! Optional: Paste YTMUSIC_SESSION cookie for personalization.")
    cookies_str = st.text_area("Cookies (JSON optional)", placeholder='{}', height=100)
    try:
        cookies = eval(cookies_str) if cookies_str else None
    except:
        cookies = None
    yt_client = get_ytmusic_client(cookies)

    if yt_client:
        st.success("✅ YouTube Music connected! Real videos.")
    else:
        st.warning("❌ Using basic search (falls back to samples if error).")

    st.header(" Quick Eval")
    st.write(f"**Text Model Acc:** {acc:.2%} (on 100 samples)")
    st.info("**Tech:** DistilRoBERTa (text), DeepFace (face, FIXED), YouTube Music Search (recs).")
    st.write("**Datasets:** dair-ai/emotion (text), YouTube Music (music).")
    st.success("Deployed via Ngrok—share this URL!")

# Tabs
tab1, tab2 = st.tabs([" Text Input", " Upload Face Image"])

with tab1:
    st.subheader("Enter your thoughts:")
    user_text = st.text_area("What's on your mind?", height=150, placeholder="E.g., 'I'm so excited about the weekend!'")

    if st.button("Analyze Text Mood", type="primary"):
        if user_text:
            with st.spinner("Detecting..."):
                emotion, confidence, scores = detect_text_emotion(user_text)

            col1, col2 = st.columns(2)
            with col1:
                st.metric("Detected Mood", emotion.upper(), delta=f"{confidence:.1%}")
            with col2:
                if st.checkbox("Show Breakdown"):
                    scores_df = pd.DataFrame([{"Emotion": k.title(), "Score": v} for k, v in scores.items()]).sort_values("Score", ascending=False)
                    st.dataframe(scores_df.style.format({"Score": "{:.2%}"}), use_container_width=True)

            st.subheader("🎵 Recommended Songs (YouTube Music):")
            recs = get_music_recs(emotion, yt_client)
            for rec in recs:
                col_a, col_b = st.columns([3, 1])
                with col_a:
                    st.write(f"• **{rec['title']}** by {rec['artist']}  (Mood: {rec['mood']}, Valence: {rec['valence']:.2f})")
                with col_b:
                    if rec.get('video_url'):
                        st.video(rec['video_url'])

with tab2:
    st.subheader("Upload a face image (clear, front-facing for best results):")
    uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "png", "jpeg"])

    if uploaded_file is not None:
        image = Image.open(uploaded_file)
        st.image(image, caption="Uploaded Image", use_column_width=True)

        if st.button("Detect Face Emotion", type="primary"):
            with st.spinner("Analyzing face..."):
                img_byte_arr = io.BytesIO()
                image.save(img_byte_arr, format='PNG')
                img_bytes = img_byte_arr.getvalue()

                # FIXED: Convert bytes to PIL Image
                image_pil = Image.open(io.BytesIO(img_bytes)).convert('RGB')
                emotion, confidence, scores = detect_face_emotion(image_pil)

            col1, col2 = st.columns(2)
            with col1:
                st.metric("Detected Mood", emotion.title(), delta=f"{confidence:.1%}")
            with col2:
                if st.checkbox("Show Breakdown"):
                    scores_df = pd.DataFrame([{"Emotion": k.title(), "Score": v} for k, v in scores.items()]).sort_values("Score", ascending=False)
                    st.dataframe(scores_df.style.format({"Score": "{:.2%}"}), use_container_width=True)

            st.subheader("🎵 Recommended Songs (YouTube Music):")
            recs = get_music_recs(emotion, yt_client)
            for rec in recs:
                col_a, col_b = st.columns([3, 1])
                with col_a:
                    st.write(f"• **{rec['title']}** by {rec['artist']}  (Mood: {rec['mood']}, Valence: {rec['valence']:.2f})")
                with col_b:
                    if rec.get('video_url'):
                        st.video(rec['video_url'])
'''

with open('app.py', 'w') as f:
    f.write(app_code)
print("✅ app.py updated.")

✅ app.py updated! Face detection fixed (bytes → PIL), YouTube embeds ready.


In [17]:
from pyngrok import ngrok
!ngrok authtoken 340cqYkCheSe4WPgFAX1nvwhnAJ_3cnHmobEtWKHDKadqHPJC  # Token

ngrok.kill()

import subprocess
import threading
import time

def run_streamlit():
    subprocess.run(["streamlit", "run", "app.py", "--server.port", "8501", "--server.address", "0.0.0.0"])

thread = threading.Thread(target=run_streamlit)
thread.start()
time.sleep(10)

public_url = ngrok.connect(8501)
print(f"🌐 Public URL: {public_url}")
print("App live! Test with mood input—YouTube videos embed & play.")

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
🌐 Public URL: NgrokTunnel: "https://discreet-francene-gravest.ngrok-free.dev" -> "http://localhost:8501"
App live! Test with mood input—YouTube videos embed & play.
